Cell 0: Project root

In [1]:
from pathlib import Path
import os
import sys

# Resolve project root from notebook location
PROJECT_ROOT = Path(os.getcwd()).resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

# Make sure src/ is importable
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print("PROJECT_ROOT:", PROJECT_ROOT)

PROJECT_ROOT: C:\Users\adamc\Documents\Projects\GeoAI-LandUse-Model


Cell 1: Paths + Imports

In [1]:
import sys, os
from pathlib import Path

PROJECT_ROOT = Path(os.getcwd()).resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print("Project root:", PROJECT_ROOT)

from src.train import train_model
from src.dataset import load_eurosat_dataset
from src.models import build_model
from src.eval import evaluate_model

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

DATA_DIR = PROJECT_ROOT / "data" / "raw" / "EuroSAT_RGB"
print("Data dir:", DATA_DIR)

Project root: C:\Users\adamc\Documents\Projects\GeoAI-LandUse-Model
Device: cpu
Data dir: C:\Users\adamc\Documents\Projects\GeoAI-LandUse-Model\data\raw\EuroSAT_RGB


Cell 2: Phase 1 Train

In [2]:
model, class_names = train_model(
    data_dir=DATA_DIR,
    model_name="resnet18",
    pretrained=True,
    freeze_backbone=True,   # train only the head
    epochs=8,
    lr=1e-3,
    batch_size=64,
    img_size=64,
    seed=42,
    aug_level="light",
    weight_decay=1e-4,
    label_smoothing=0.05,
    early_stop_patience=3,
)

Using device: cpu


c:\Users\adamc\Documents\Projects\GeoAI-LandUse-Model\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\adamc\Documents\Projects\GeoAI-LandUse-Model\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/8: 100%|██████████| 296/296 [04:58<00:00,  1.01s/it]


Epoch 1 | Train Loss: 1.1517 | Val Acc: 0.7674 | LR: 1.00e-03
Saved BEST model so far to C:\Users\adamc\Documents\Projects\GeoAI-LandUse-Model\models\simple_cnn_v2_best.pth (val_acc=0.7674)


Epoch 2/8: 100%|██████████| 296/296 [01:08<00:00,  4.32it/s]


Epoch 2 | Train Loss: 0.9176 | Val Acc: 0.7884 | LR: 1.00e-03
Saved BEST model so far to C:\Users\adamc\Documents\Projects\GeoAI-LandUse-Model\models\simple_cnn_v2_best.pth (val_acc=0.7884)


...

Epoch 6 | Train Loss: 0.8613 | Val Acc: 0.8027 | LR: 1.00e-03
Saved BEST model so far to C:\Users\adamc\Documents\Projects\GeoAI-LandUse-Model\models\simple_cnn_v2_best.pth (val_acc=0.8027)


Epoch 7/8: 100%|██████████| 296/296 [00:58<00:00,  5.02it/s]


Epoch 7 | Train Loss: 0.8546 | Val Acc: 0.8072 | LR: 1.00e-03
Saved BEST model so far to C:\Users\adamc\Documents\Projects\GeoAI-LandUse-Model\models\simple_cnn_v2_best.pth (val_acc=0.8072)


Epoch 8/8: 100%|██████████| 296/296 [01:03<00:00,  4.68it/s]


Epoch 8 | Train Loss: 0.8453 | Val Acc: 0.8025 | LR: 1.00e-03
Done. Best Val Acc: 0.8072


Cell 3: Evaluate training

In [2]:
from pathlib import Path
import torch

models_dir = PROJECT_ROOT / "models"
MODEL_PATH = models_dir / "resnet18_pre_frozen_best.pth"

if not MODEL_PATH.exists():
    print("Missing checkpoint:", MODEL_PATH)
    print("Available .pth files:")
    for p in sorted(models_dir.glob("*.pth")):
        print(" -", p.name)
    raise FileNotFoundError(f"Checkpoint not found: {MODEL_PATH}")

print("Loading:", MODEL_PATH)

model = build_model(
    num_classes=len(class_names),
    model_name="resnet18",
    pretrained=True,
    freeze_backbone=False,
).to(device)

model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

test_acc, cm, report = evaluate_model(model, test_loader, class_names, device)
print("Test Accuracy:", round(test_acc, 4))
print(report)

Missing checkpoint: C:\Users\adamc\Documents\Projects\GeoAI-LandUse-Model\models\resnet18_pre_frozen_best.pth
Available .pth files:
 - simple_cnn.pth
 - simple_cnn_augmented.pth
 - simple_cnn_v2.pth
 - simple_cnn_v2_best.pth
 - simple_cnn_v2_last.pth


FileNotFoundError: Checkpoint not found: C:\Users\adamc\Documents\Projects\GeoAI-LandUse-Model\models\resnet18_pre_frozen_best.pth

Cell 4: Confusion matrices

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

fig_dir = PROJECT_ROOT / "reports" / "figures"
fig_dir.mkdir(parents=True, exist_ok=True)

cm_np = np.array(cm)

# Normalize rows - each true class sums to 1
cm_norm = cm_np / np.maximum(cm_np.sum(axis=1, keepdims=True), 1)

plt.figure(figsize=(10, 10))
plt.imshow(cm_norm, vmin=0, vmax=1)
plt.xticks(ticks=np.arange(len(class_names)), labels=class_names, rotation=90)
plt.yticks(ticks=np.arange(len(class_names)), labels=class_names)
plt.title("Confusion Matrix (Test Set) - Normalized")
plt.colorbar()
plt.tight_layout()

out_path = fig_dir / "confusion_matrix_resnet18_normalized.png"
plt.savefig(out_path, dpi=200, bbox_inches="tight")
print("Saved:", out_path)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

fig_dir = PROJECT_ROOT / "reports" / "figures"
fig_dir.mkdir(parents=True, exist_ok=True)

cm_np = np.array(cm)

plt.figure(figsize=(10, 10))
plt.imshow(cm_np)
plt.xticks(ticks=np.arange(len(class_names)), labels=class_names, rotation=90)
plt.yticks(ticks=np.arange(len(class_names)), labels=class_names)
plt.title("Confusion Matrix (Test Set) – Raw Counts")
plt.colorbar()
plt.tight_layout()

out_path = fig_dir / "confusion_matrix_resnet18_raw.png"
plt.savefig(out_path, dpi=200, bbox_inches="tight")
print("Saved:", out_path)

plt.show()

Cell 5: Fine-tune

In [ ]:
model, class_names = train_model(
    data_dir=DATA_DIR,
    model_name="resnet18",
    pretrained=True,
    freeze_backbone=False,   # fine-tune
    epochs=10,
    lr=1e-4,                 # lower LR for fine-tuning
    batch_size=64,
    img_size=64,
    seed=42,
    aug_level="light",
    weight_decay=1e-4,
    label_smoothing=0.05,
    early_stop_patience=3,
)